# Google DATABASE

[Google DATABASE](https://cloud.google.com/DATABASE).

Load documents from `DATABASE`.

## Pre-reqs

In [ ]:
%pip install PACKAGE_NAME

In [3]:
from PACKAGE import LOADER

## Basic Usage

### Load from table

In [6]:
loader = LOADER()

data = loader.load()

### Load from query

In [ ]:
loader = LOADER()

data = loader.load()

## Customize Document Page Content & Metadata

In [8]:
loader = LOADER()

data = loader.load()

### Customize Page Content Format

## Save Documents to table

In [ ]:
saver = SAVER()
saver.add_documents(docs)

## Customize Connection & Authentication

In [ ]:
from google.cloud.DATABASE import Client

creds = ""
client = Client(creds=creds)
loader = LOADER(
    client=client,
)